# Computer Vision CNNs Notebook

This notebook contains exercises for the computer vision CNNs material.

In [13]:
import tensorflow as tf
import pickle, bz2
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


In [3]:
!python -m wget https://github.com/shegenbart/Jupyter-Exercises/raw/main/data/kth_tips.pbz2 -o kth_tips.pbz2


Saved under kth_tips (1).pbz2


In [43]:
with bz2.BZ2File('./kth_tips.pbz2', 'rb') as fd:
    kth_dataset = pickle.load(fd)

X_train, X_test, Y_train, Y_test = train_test_split(kth_dataset['X'], kth_dataset['Y'], random_state=12345)
Y_train = tf.keras.utils.to_categorical(Y_train)
Y_test = tf.keras.utils.to_categorical(Y_test)

X_train = X_train / 255
X_test = X_test / 255

### Exercise 1 - Classify Image Dataset with Conventional Neural Network

I have provided you with an image dataset (the same we used in the Local Binary Patterns exercise). Your images are in
X_train, X_test while youre labels are in Y_train, Y_test.

Your job is to train a neural network on the pixel values to classify the images. To do so:
* Reshape the images from $(n,200,200)$ to $(n,40000)$
* Create a neural network model using keras
* Train your neural network on the (X_train, Y_train) and validate on the (X_test, Y_test) data.
* Compare your results with the LBP-results (if you have them)
* Explain what happened




In [44]:
print(X_train.shape, Y_train.shape, X_test.shape, Y_test.shape)

# 1) Reshape the images from (n,200,200) to (n,40000)
X_train = X_train.reshape(X_train.shape[0], 40000)
X_test = X_test.reshape(X_test.shape[0], 40000)

print(X_train.shape, Y_train.shape, X_test.shape, Y_test.shape)

(607, 200, 200) (607, 10) (203, 200, 200) (203, 10)
(607, 40000) (607, 10) (203, 40000) (203, 10)


In [51]:
# 2) Create a neural network model with keras
nn = tf.keras.models.Sequential()
nn.add(Dense(512, input_dim=40000, activation='relu'))
nn.add(Dense(256, activation='relu'))
nn.add(Dense(10, activation='relu'))

In [55]:
# 3) train the neural network on the (X_train, Y_train) data and validate on (X_test, Y_test)
nn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
nn.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=25, batch_size=32)

Epoch 1/25
19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 63ms/step - accuracy: 0.1361 - loss: 7.6294 - val_accuracy: 0.1281 - val_loss: 7.0174
Epoch 2/25
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 58ms/step - accuracy: 0.1091 - loss: 7.4997 - val_accuracy: 0.0985 - val_loss: 7.0139
Epoch 3/25
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 56ms/step - accuracy: 0.0902 - loss: 7.6250 - val_accuracy: 0.1084 - val_loss: 7.0131
Epoch 4/25
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 56ms/step - accuracy: 0.1270 - loss: 7.8557 - val_accuracy: 0.0985 - val_loss: 7.0159
Epoch 5/25
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 56ms/step - accuracy: 0.1061 - loss: 7.5463 - val_accuracy: 0.0936 - val_loss: 7.0149
Epoch 6/25
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 56ms/step - accuracy: 0.1120 - loss: 7.8742 - val_accuracy: 0.1182 - val_loss: 7.0108
Epoch 7/25
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 56ms/step - accuracy: 0.1086 - loss: 7.8715 - val_accuracy: 0.1034 - val_loss: 7.0117
Epoch 8/25
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 56ms/step - accuracy: 0.1184 - loss: 7.6587 - val_accuracy: 0.1133 - v

In [56]:
_, accuracy = nn.evaluate(X_test, Y_test)
print('Accuracy: %.2f' % (accuracy*100))

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1944 - loss: 7.7934 
Accuracy: 16.26


### Exercise 2: Convolutional Layer

In the cell below, I have provided you with a very simple CNN consisting only of a single convolutional layer.
You can access the weights (kernels) of a keras model by accessing the layer with:

```python
kernel_list = cnn.layers[0].get_weights() # This gives us a list of kernels in the layer
```


Experiment with different parameters for ```filters, kernel_size``` and ```input_shape``` and have
a look at the kernels to get a solid understanding of how these kernels a CNN uses look like.

Answer the following questions:
* How are the kernel dimensions influenced by the input dimensions?
* How is the number of parameters in the network influenced by the input dimension, the kernel size and the number of filters?
* Explain and summarize your findings.

In [59]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, InputLayer

In [60]:
cnn = Sequential()
cnn.add(InputLayer(input_shape=(128,128,3)))
cnn.add(Conv2D(filters=1, kernel_size=(5,5)))
cnn.add(Conv2D(filters=2, kernel_size=(3,3)))

cnn.summary()

Model: "sequential_20"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_2 (Conv2D)               │ (None, 124, 124, 1)    │            76 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 122, 122, 2)    │            20 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 96 (384.00 B)

 Trainable params: 96 (384.00 B)

 Non-trainable params: 0 (0.00 B)

### Exercise 3 - Pooling Layer

Create a convolutional neural network with two convolution layers followed by pooling layers (use either max pooling or average pooling).

Experiment with different values for ```pool_size, strides``` and study how the output shape is affected.
Answer the following questions:
* What is the difference in output size between max pooling and average pooling? Why?
* What is the impact on the receptive field of a convolutional layer following a pooling layer?
* How is the amount of parameters affected by a pooling layer?


### Exercise 4: Flatten and GlobalAveragePooling

In this exercise we will study two methods we can use to transfrom multi-dimensional feature maps to one-dimensional feature maps as used as input for Dense layers.

1. Create a convolution neural network using two convolutional layers.
    * Use a Flatten layer to transform your multi-dimensional feature map to a one-dimensional feature map.
    * Add a Dense output layer for a classification problem with 10 classes.
    * Where is the majority of the weights in your network architecture?
    * How could this influence the training of your network?

2. Create a convolution neural network using two convolutional layers.
    * Use a GlobalAveragePooling2D layer to transform your multi-dimensional feature map to a one-dimensional feature map.
    * Create a network architecture for a classification problem with 10 classes without using a Dense layer.
    * Where is the majority of weights in your network architecture?
    * How could this influence the training of your network?

    


### Exercise 5: Classification using CNNS


In the first exercise in this notebook we already downloaded and prepared the KTH-TIPS dataset we used previously for classification. We saw that using a conventional NN
did not work very well in this scenario.

1. Create a CNN of your choice to train a classifier for the provided dataset.
    * Train on (X_train, Y_train), Validate on (X_test, Y_test) (you can specify validation_data in the ```model.fit()``` function of keras)
2. Compare using Flatten() and GlobalAveragePooling() for feeding your features into Dense layers
    * How does it effect the number of parameters, the accuracy of your results and the time used to train the models?
3. What is the highest accuracy you can reach with the smallest amount of parameters?


